<a href="https://colab.research.google.com/github/mralamdari/YOLO/blob/main/Yolo_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
#clone OIDv4_ToolKit
!git clone https://github.com/mralamdari/OIDv4_ToolKit.git

[Errno 2] No such file or directory: '/content/darknet'
/content
Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 499 (delta 32), reused 52 (delta 31), pack-reused 444
Receiving objects: 100% (499/499), 34.11 MiB | 16.39 MiB/s, done.
Resolving deltas: 100% (189/189), done.


In [ ]:
#install necessary libraries
# !pip3 install -r requirements.txt
!pip install awscli

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1


In [ ]:
#Move to OIDv4_ToolKit folder
%cd OIDv4_ToolKit

#Mention the number of objects you want to train
NUM_CLASSES=2

#Write your desired objects (--classes) and data type (--type_csv)  and the number of images you want (--limit)
!python3 main.py downloader --classes Person Car --type_csv train --limit 500

/content/OIDv4_ToolKit

		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Backpack.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] 